# W12 – Time Series Forecasting Demo with PJM Hourly Load

This notebook reproduces the key **time series concepts** from the W12 lecture
using the **PJM West hourly electricity load** dataset (`PJMW_hourly.csv`).

It follows the same structure as the lecture slides:
1. Time series plot
2. Decomposition into trend / seasonality / residuals (MSTL with multiple seasonalities)
3. Moving-average detrending
4. Seasonality indices (e.g., hour-of-day and day-of-week)
5. Rolling window features
6. Point forecast vs prediction intervals
7. One-step vs multi-step forecasting
8. Simple naive forecast example

> **Note:** Place `PJMW_hourly.csv` in the same folder as this notebook before running.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
try:
    # MSTL is available in newer statsmodels versions
    from statsmodels.tsa.seasonal import MSTL
    HAS_MSTL = True
except ImportError:
    HAS_MSTL = False

%matplotlib inline

# === Load PJM West hourly load dataset ===
# Make sure PJMW_hourly.csv is in the same directory as this notebook.
df = pd.read_csv("PJMW_hourly.csv")  # columns: Datetime, PJMW_MW
df['Datetime'] = pd.to_datetime(df['Datetime'])
df = df.sort_values('Datetime')
df = df.set_index('Datetime')

# Use a single univariate series
ts = df['PJMW_MW'].asfreq('H')

ts.head()


## 1. Time Series Plot (Hourly Load)

This corresponds to the *time series plot* slides. We first look at the
full series, then zoom into a shorter period (e.g., one month) to see the
daily pattern more clearly.

In [ ]:

# Full series
plt.figure(figsize=(12,4))
plt.plot(ts)
plt.title('PJM West Hourly Load (Full Series)')
plt.xlabel('Datetime')
plt.ylabel('Load (MW)')
plt.tight_layout()

# Zoom into one month for readability
one_month = ts.loc[ts.index.min(): ts.index.min() + pd.Timedelta(days=30)]
plt.figure(figsize=(12,4))
plt.plot(one_month)
plt.title('PJM West Hourly Load – First 30 Days')
plt.xlabel('Datetime')
plt.ylabel('Load (MW)')
plt.tight_layout()


## 2. Decomposition with Multiple Seasonalities (MSTL / STL)

We automatically **check candidate seasonal periods** that are common in
hourly electricity data:

* 24 hours (daily pattern)
* 24 × 7 = 168 hours (weekly pattern)
* 24 × 365 ≈ 8760 hours (yearly pattern, only if we have enough data)

We keep those periods that:
1. Are shorter than half of the series length, and
2. Have reasonably high autocorrelation at that lag.

Then we use **MSTL** if available; otherwise we fall back to standard STL
with just the strongest period.

In [ ]:

from statsmodels.tsa.stattools import acf

# Candidate seasonal periods (in hours)
candidate_periods = [24, 24*7, 24*365]

# Compute autocorrelation up to the max candidate
max_lag = min(max(candidate_periods), len(ts) - 2)
acf_vals = acf(ts.dropna(), nlags=max_lag, fft=True)

detected_periods = []
for p in candidate_periods:
    if p < len(ts) / 2:
        rho = acf_vals[p]
        print(f"Period {p} (hours), autocorr = {rho:.3f}")
        if rho > 0.3:  # simple threshold for 'strong' seasonality
            detected_periods.append(p)

print("\nDetected seasonal periods (hours):", detected_periods)

# Use MSTL if possible and we have at least one detected period
if HAS_MSTL and len(detected_periods) > 0:
    mstl = MSTL(ts, periods=detected_periods)
    res = mstl.fit()
    fig = res.plot()
    fig.set_size_inches(12, 8)
    plt.tight_layout()
else:
    # Fallback: use STL with the strongest candidate (e.g., daily)
    if len(detected_periods) == 0:
        period = 24  # default to daily
    else:
        period = detected_periods[0]
    print(f"Using STL with period={period} as fallback.")
    stl = STL(ts, period=period)
    res = stl.fit()
    fig = res.plot()
    fig.set_size_inches(12, 8)
    plt.tight_layout()


## 3. Moving-Average Detrending

We apply a **moving average** to smooth short-term fluctuations and
highlight the underlying trend. For hourly load, a common choice is a
`24-hour` (daily) or `168-hour` (weekly) moving average.

In [ ]:

ma_24 = ts.rolling(window=24, center=False).mean()
ma_168 = ts.rolling(window=168, center=False).mean()

plt.figure(figsize=(12,4))
plt.plot(ts, label='Original', alpha=0.5)
plt.plot(ma_24, label='24-hour MA')
plt.plot(ma_168, label='168-hour MA')
plt.xlabel('Datetime')
plt.ylabel('Load (MW)')
plt.title('Moving Average Smoothing (24h and 168h)')
plt.legend()
plt.tight_layout()


## 4. Seasonality Indices (Hour-of-Day and Day-of-Week)

We compute **period-adjusted averages** similar to the lecture example:
* Average load by **hour of day** (0–23)
* Convert to a multiplicative **seasonality index** by dividing by the
  overall mean.

This shows which hours are above/below the typical load level.

In [ ]:

df_season = ts.to_frame('load')
df_season['hour'] = df_season.index.hour
df_season['dow'] = df_season.index.dayofweek  # 0=Mon, 6=Sun

# Hour-of-day seasonality
hour_mean = df_season.groupby('hour')['load'].mean()
overall_mean = hour_mean.mean()
hour_index = hour_mean / overall_mean

print('Overall mean load:', overall_mean)
display(hour_index.to_frame('hour_season_index'))

plt.figure(figsize=(8,4))
plt.plot(hour_index.index, hour_index.values, marker='o')
plt.xlabel('Hour of day')
plt.ylabel('Seasonality index')
plt.title('Hour-of-day Seasonality Index (Multiplicative)')
plt.tight_layout()

# Day-of-week seasonality (optional extra)
dow_mean = df_season.groupby('dow')['load'].mean()
dow_index = dow_mean / dow_mean.mean()

plt.figure(figsize=(8,4))
plt.plot(dow_index.index, dow_index.values, marker='o')
plt.xticks(range(7), ['Mon','Tue','Wed','Thu','Fri','Sat','Sun'])
plt.xlabel('Day of week')
plt.ylabel('Seasonality index')
plt.title('Day-of-week Seasonality Index (Multiplicative)')
plt.tight_layout()


## 5. Rolling Window Feature Engineering

We create a **24-hour rolling mean** as an example of a rolling window
feature that could be used as an input to a forecasting model.

In [ ]:

df_feat = ts.to_frame('load')
df_feat['roll_mean_24'] = df_feat['load'].rolling(window=24).mean()

df_feat.tail()

# Visualize a short period with the rolling feature
sample = df_feat.iloc[:24*14]  # first two weeks
plt.figure(figsize=(12,4))
plt.plot(sample.index, sample['load'], label='Load')
plt.plot(sample.index, sample['roll_mean_24'], label='24h rolling mean')
plt.xlabel('Datetime')
plt.ylabel('Load (MW)')
plt.title('Rolling Window Feature (First 2 Weeks)')
plt.legend()
plt.tight_layout()


## 6. Point Forecast vs Prediction Interval

To illustrate **point forecasts** vs **prediction intervals** (as in the
lecture), we build a simple baseline:

* Forecast horizon: next 24 hours
* Point forecast: last observed load (constant forecast)
* Interval: ±10% around the point forecast (for illustration only).

In [ ]:

horizon = 24
last_value = ts.iloc[-1]

point_forecast = np.repeat(last_value, horizon)
lower = point_forecast * 0.9
upper = point_forecast * 1.1

x = np.arange(1, horizon+1)
plt.figure(figsize=(8,4))
plt.plot(x, point_forecast, marker='o', label='Point forecast')
plt.fill_between(x, lower, upper, alpha=0.3, label='Prediction interval (±10%)')
plt.xlabel('Forecast step (hours ahead)')
plt.ylabel('Load (MW)')
plt.title('Point Forecast vs Prediction Interval (Simple Baseline)')
plt.legend()
plt.tight_layout()


## 7. One-step vs Multi-step Forecasting (Conceptual Demo)

Here we simulate **error growth** for one-step-ahead vs multi-step-ahead
forecasting. The numbers are toy values meant to visualize the concept
from the slides, not calibrated to this dataset.

In [ ]:

steps = np.arange(1, 13)  # up to 12 hours ahead

# Simulated RMSE curves
one_step_rmse = 5 + 0.2*(steps-1)       # slowly increasing
multi_step_rmse = 5 + 0.6*(steps-1)     # faster increase (error accumulation)

plt.figure(figsize=(8,4))
plt.plot(steps, one_step_rmse, marker='o', label='One-step ahead')
plt.plot(steps, multi_step_rmse, marker='o', label='Multi-step ahead')
plt.xlabel('Forecast horizon (hours ahead)')
plt.ylabel('Simulated RMSE')
plt.title('Error Growth: One-step vs Multi-step Forecasting')
plt.legend()
plt.tight_layout()


## 8. Naive Forecast Baseline (Train/Test Split)

We reserve the **last 7 days (168 hours)** as a test set and fit a simple
**naive model**: all future values equal the **last training observation**.
This gives a baseline for future, more sophisticated models.

In [ ]:

test_horizon = 24 * 7  # last week
train = ts.iloc[:-test_horizon]
test = ts.iloc[-test_horizon:]

naive_forecast = np.repeat(train.iloc[-1], test_horizon)

plt.figure(figsize=(12,4))
plt.plot(test.index, test.values, label='Actual')
plt.plot(test.index, naive_forecast, label='Naive forecast')
plt.xlabel('Datetime')
plt.ylabel('Load (MW)')
plt.title('Naive Forecast vs Actual (Last 7 Days)')
plt.legend()
plt.tight_layout()
